In [ ]:
################################## KENNETH EKPETERE #################################
################################# PMP IMPLEMENTATION ###############################
##################################### (C) 2024  ########################################

In [2]:
import time
import pandas as pd
import numpy as np
import sys
from datetime import date
from datetime import datetime, timedelta
import math
import statistics
from math import exp
from scipy.stats import tmean, tstd

#### **HERSHFIELD PMP**

In [3]:
# Manage files
input_file = "stn_max.csv"
output_file = "PMP_Output.csv"

data = pd.read_csv(input_file, float_precision='round_trip')

# Group data by unique ID
grouped = data.groupby('ID')

# Initialize empty list to hold DataFrame chunks (we need this to store the output values)
output_chunks = []

# Loop through each group
for name, group in grouped:
    # Compute maximum value for each column
    max_pre = group[['pre30m', 'pre1h', 'pre2h', 'pre3h', 'pre6h', 'pre12h', 'pre24h', 'pre48h', 'pre72h']].max()
    
    # Compute mean for each column
    mean_pre = group[['pre30m', 'pre1h', 'pre2h', 'pre3h', 'pre6h', 'pre12h', 'pre24h', 'pre48h', 'pre72h']].mean()
    
    # Compute standard deviation for each column
    std_pre = group[['pre30m', 'pre1h', 'pre2h', 'pre3h', 'pre6h', 'pre12h', 'pre24h', 'pre48h', 'pre72h']].std()
    
    # Compute trimmed mean for each column
    trimmed_mean = group[['pre30m', 'pre1h', 'pre2h', 'pre3h', 'pre6h', 'pre12h', 'pre24h', 'pre48h', 'pre72h']].apply(lambda x: np.mean(np.sort(x)[:-1]))
    
    # Compute trimmed standard deviation for each column
    trimmed_std = group[['pre30m', 'pre1h', 'pre2h', 'pre3h', 'pre6h', 'pre12h', 'pre24h', 'pre48h', 'pre72h']].apply(lambda x: np.std(np.sort(x)[:-1], ddof=1))
    
    # Compute frequency factor for each column
    freqfact = ((max_pre - trimmed_mean) / trimmed_std)
    
    # Compute HPMP for each column
    HPMP = mean_pre + (freqfact * std_pre)
    
    # Create DataFrame chunk for this group
    output_chunk = pd.DataFrame({
        'ID': name,
        'pre_HPMP30m': HPMP['pre30m'],
        'pre_HPMP1h': HPMP['pre1h'],
        'pre_HPMP2h': HPMP['pre2h'],
        'pre_HPMP3h': HPMP['pre3h'],
        'pre_HPMP6h': HPMP['pre6h'],
        'pre_HPMP12h': HPMP['pre12h'],
        'pre_HPMP24h': HPMP['pre24h'],
        'pre_HPMP48h': HPMP['pre48h'],
        'pre_HPMP72h': HPMP['pre72h']
    }, index=[0])  # Ensure each chunk has only one row
    
    # Append chunk to list
    output_chunks.append(output_chunk)

# Concatenate all chunks into final DataFrame
output_data = pd.concat(output_chunks, ignore_index=True)

# Write output DataFrame to CSV
output_data.to_csv(output_file, index=False)
print("PMP calculations completed")


pre_HPMP calculations completed
